<div align="center">
  <a href="http://www.sharif.edu/">
    <img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" alt="SUT Logo" width="140">
  </a>
  
  # Sharif University of Technology
  ### Electrical Engineering Department

  ## Signals and Systems
  #### *Final Project - Spring 2025*
</div>

---

<div align="center">
  <h1>
    <b>Object Tracker</b>
  </h1>
  <p>
    An object tracking system using YOLO for detection and various algorithms (KCF, CSRT, MOSSE) for tracking.
  </p>
</div>

<br>

| Professor                  |
| :-------------------------: |
| Dr. Mohammad Mehdi Mojahedian |

<br>

| Contributors              |
| :-----------------------: |
| **Amirreza Mousavi** |
| **Mahdi Falahi** |
| **Zahra Miladipour** |


## 1: Preparing The Materials


In [85]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import time

1.1 : Calculating HOG ( return the hog of the image)
 

In [86]:
def hog_scaling (image):
    img = cv2.cvtColor(image ,cv2.COLOR_BGR2GRAY)
    filter = cv2.HOGDescriptor((64, 64), (16, 16), (8, 8), (8, 8), 9)
    resized_image = cv2.resize(img, (64, 64))
    result = filter.compute(resized_image)
    return result if result is not None else np.zeros((hog_descriptor.getDescriptorSize(1764),))


In [87]:
def hog_channel(image):
    img = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
    win_size = (image.shape[1], image.shape[0])
    filter = cv2.HOGDescriptor(win_size , (16 ,16 ) , (8,8) , (8,8) , 9)
    result = filter.compute(img)
    height =  (win_size[1] - 16) // 8 + 1    # 8 is the block strid(x) we can change that consider the trade off of time _ accuracy
    width = (win_size[0] - 16) // 8 + 1   # 8 is the block strid(y) we can change that consider the trade off of time _ accuracy
    features_per_block = 2 * 2 * 9
    hog_features = result.reshape((height, width, features_per_block))
    return cv2.resize(hog_features, (win_size[0] // 8, win_size[1] // 8))


1.2 : Checking The Scale (return the scale of the image in the current frame)

In [88]:
def scaled_check(frame, pos, base_size, scale_factors, scale_model_A, scale_model_B, lambda_trust=0.01):
    scale_features = []
    for scale in scale_factors:
        w_s, h_s = int(base_size[0] * scale), int(base_size[1] * scale)
        x_s, y_s = int(pos[0] - w_s / 2), int(pos[1] - h_s / 2)
        patch_s = frame[y_s:y_s+h_s, x_s:x_s+w_s]
        if patch_s.shape[0] < 16 or patch_s.shape[1] < 16:
            scale_features.append(np.zeros_like(scale_features[0]) if len(scale_features) > 0 else np.zeros((1764,)))
            continue
        resized = cv2.resize(patch_s, (64, 64))
        scale_features.append(hog_scaling(resized))
    
    SF = np.fft.fft(np.array(scale_features), axis=0)
    scale_H = scale_model_A / (scale_model_B[:, np.newaxis] + lambda_trust)
    response_f = np.sum(np.conj(scale_H) * SF, axis=1)
    response = np.real(np.fft.ifft(response_f))
    
    best_scale_idx = np.argmax(response)
    return scale_factors[best_scale_idx]

1.3 : Prediction The Next Frame's Center ( Kalman Filter )

In [89]:
def kalman_prediction( F , X_k_1 , P_k_1 , Q_k):
    x_k = np.dot(F , X_k_1)
    p_k = np.dot( F , np.dot(P_k_1 , F.T)) + Q_k
    return x_k , p_k

In [90]:
def kalman_updating(x_k , p_k , H_k , z_k , R_k):
    k_1 = np.dot(np.dot(H_k , p_k) , H_k.T) + R_k
    k_2 = np.dot(p_k , H_k.T)
    K = np.dot(k_2 , np.linalg.inv(k_1))
    P_k_new = p_k - np.dot(np.dot(K , H_k) , p_k)
    x_k_new = x_k + np.dot(K , (z_k - np.dot(H_k , x_k)))
    return x_k_new , P_k_new

1.4 : Updating Method

In [91]:
def filter_updating(H_new , H_old , alpha):
    result = alpha * H_new + (1-alpha) * H_old
    return result

1.5 : Finding The Channels

In [92]:
def extract_channels(image):
    hog_features = hog_channel(image)
    colors = np.array([
        [0.00, 0.00, 0.00], [45.37, -4.33, -33.43], [43.08, 17.51, 37.53],
        [53.59, 0.00, 0.00], [47.31, -45.33, 41.35], [65.75, 71.45, 63.32],
        [76.08, 22.25, -21.46], [32.30, 79.19, -107.86], [52.23, 75.43, 37.36],
        [100.00, 0.00, 0.00], [92.13, -16.53, 93.35]
    ], dtype=np.float32)
    
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    pixels = image_lab.reshape(-1, 3).astype(np.float32)
    distances = np.sum((pixels[:, np.newaxis, :] - colors[np.newaxis, :, :]) ** 2, axis=2)
    closest_color_indices = np.argmin(distances, axis=1)
    
    cn_features_flat = np.zeros((pixels.shape[0], colors.shape[0]), dtype=np.float32)
    cn_features_flat[np.arange(pixels.shape[0]), closest_color_indices] = 1
    cn_features = cn_features_flat.reshape(image.shape[0], image.shape[1], -1)

    hog_resized = cv2.resize(hog_features, (image.shape[1], image.shape[0]))
    return np.dstack((hog_resized, cn_features))

1.6 : Teaching The Filters

In [93]:
def teaching ( f , g , lambda_trust):
    X = np.fft.fft2(f , axes = (0 ,1 ))
    G = np.fft.fft2(g)
    G1 = np.expand_dims(G , axis = 2)
    num = np.conj(X) * G1
    denom = np.sum(np.conj(X) * X , axis =2) + lambda_trust
    return num, denom


In [94]:
def calculate_psr(response_map):
    if not isinstance(response_map, np.ndarray) or response_map.size == 0:
        return 0.0
    
    peak_loc = np.unravel_index(np.argmax(response_map), response_map.shape)
    peak_value = response_map[peak_loc]

    sidelobe_window = 11
    h, w = response_map.shape
    y_start = max(0, peak_loc[0] - sidelobe_window // 2)
    y_end = min(h, peak_loc[0] + sidelobe_window // 2 + 1)
    x_start = max(0, peak_loc[1] - sidelobe_window // 2)
    x_end = min(w, peak_loc[1] + sidelobe_window // 2 + 1)
    
    mask = np.ones_like(response_map, dtype=bool)
    mask[y_start:y_end, x_start:x_end] = False
    
    sidelobe = response_map[mask]
    
    if sidelobe.size > 0:
        mean_sidelobe = np.mean(sidelobe)
        std_sidelobe = np.std(sidelobe)
        if std_sidelobe > 1e-5:
            return (peak_value - mean_sidelobe) / std_sidelobe
    return 0.0

def create_gaussian_target(size, center_x, center_y, sigma=18.0):
    height, width = size
    yy, xx = np.mgrid[0:height, 0:width]
    gaussian = np.exp(-((xx - center_x)**2 + (yy - center_y)**2) / (2 * sigma**2))
    rolled_gaussian = np.roll(np.roll(gaussian, -center_y, axis=0), -center_x, axis=1)
    return rolled_gaussian

## 2 : Main Detector

2.1 : Uploading Video

In [95]:
cap = cv2.VideoCapture('person4.mp4')
model = YOLO('yolo11n.pt')
if not cap.isOpened():
    print("wrong video")

2.1 : The Main Part

In [ ]:
tracking = False
frames_since_update = 0
MAX_FRAMES_TO_SKIP = 30
RE_DETECTION_FRAME_THRESHOLD = 5 
CONFIDENCE_THRESHOLD = 0.5 # آستانه اطمینان برای تشخیص اصلی و مجدد

model_A, model_B = None, None
scale_model_A, scale_model_B = None, None
current_pos, current_size = (0, 0), (0, 0)
fixed_roi_size = (64, 128)

dt = 1/30.0 
F = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0, 0, 0, 1]], dtype=np.float32)
H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]], dtype=np.float32)
Q = np.eye(4, dtype=np.float32) * 1e-2
R = np.eye(2, dtype=np.float32) * 25
P = np.eye(4, dtype=np.float32) * 100
kalman_state = np.zeros(4, dtype=np.float32)

last_time = 0; fps = 0; psr_score = 0.0
fps_start_time = time.time(); fps_frame_count = 0

# save the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_fps = cap.get(cv2.CAP_PROP_FPS)
if output_fps == 0: output_fps = 30.0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_writer = cv2.VideoWriter('tracking_output.mp4', fourcc, output_fps, (frame_width, frame_height))


while True:
    ret, frame = cap.read()
    if not ret: break
    
    current_time = time.time()
    dt = (current_time - last_time) if last_time > 0 else 1/30.0
    last_time = current_time
    
    if not tracking:
        results = model(frame, verbose=False, classes=[0], conf=CONFIDENCE_THRESHOLD)[0]
        if len(results.boxes) > 0:
            box = results.boxes[0]
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            w, h = x2 - x1, y2 - y1
            current_pos = (x1 + w/2, y1 + h/2); current_size = (w, h)
            kalman_state = np.array([current_pos[0], 0, current_pos[1], 0], dtype=np.float32)
            patch = cv2.resize(frame[y1:y2, x1:x2], fixed_roi_size)
            features = extract_channels(patch)
            h_roi, w_roi = fixed_roi_size[1], fixed_roi_size[0]
            target_y_2d = create_gaussian_target((h_roi, w_roi), w_roi//2, h_roi//2, 18.0)
            model_A, model_B = teaching(features, target_y_2d, 0.01)
            tracking = True
            frames_since_update = 0
    else:
        F[0,1]=dt; F[2,3]=dt
        kalman_state = F @ kalman_state
        P = F @ P @ F.T + Q
        pred_pos = (kalman_state[0], kalman_state[2])
        
        w_search=int(current_size[0]*2.0); h_search=int(current_size[1]*2.0)
        x_search=int(pred_pos[0]-w_search/2); y_search=int(pred_pos[1]-h_search/2)
        search_patch = frame[max(0,y_search):y_search+h_search, max(0,x_search):x_search+w_search]
        
        if search_patch.shape[0] > 0 and search_patch.shape[1] > 0:
            resized_patch = cv2.resize(search_patch, fixed_roi_size)
            features = extract_channels(resized_patch)
            Z = np.fft.fft2(features, axes=(0, 1))
            H_filter = model_A / (np.expand_dims(model_B, axis=2) + 0.01)
            response_f = np.sum(np.conj(H_filter) * Z, axis=2)
            response = np.real(np.fft.ifft2(response_f))
            psr_score = calculate_psr(response)

            if psr_score > 5.5:
                frames_since_update = 0
                peak_y,peak_x=np.unravel_index(np.argmax(response),response.shape)
                if peak_y > fixed_roi_size[1]/2: peak_y -= fixed_roi_size[1]
                if peak_x > fixed_roi_size[0]/2: peak_x -= fixed_roi_size[0]
                dx=(peak_x/fixed_roi_size[0])*w_search; dy=(peak_y/fixed_roi_size[1])*h_search
                measured_pos=(pred_pos[0]+dx, pred_pos[1]+dy)
                z=np.array([measured_pos[0],measured_pos[1]],dtype=np.float32)
                K=P@H.T@np.linalg.inv(H@P@H.T+R); kalman_state=kalman_state+K@(z-H@kalman_state)
                P=(np.eye(4)-K@H)@P; current_pos=(kalman_state[0],kalman_state[2])
                
                x1_up=int(current_pos[0]-current_size[0]/2); y1_up=int(current_pos[1]-current_size[1]/2)
                update_patch = frame[max(0,y1_up):y1_up+int(current_size[1]), max(0,x1_up):x1_up+int(current_size[0])]
                if update_patch.shape[0]>0 and update_patch.shape[1]>0:
                    resized_patch_up=cv2.resize(update_patch, fixed_roi_size)
                    features_new=extract_channels(resized_patch_up)
                    target_y_2d=create_gaussian_target(fixed_roi_size[::-1], fixed_roi_size[0]//2, fixed_roi_size[1]//2, 18.0)
                    new_A, new_B = teaching(features_new, target_y_2d, 0.01)
                    model_A = filter_updating(new_A, model_A, 0.025)
                    model_B = filter_updating(new_B, model_B, 0.025)
            else:
                frames_since_update += 1

            if RE_DETECTION_FRAME_THRESHOLD < frames_since_update <= MAX_FRAMES_TO_SKIP:
                x,y,w,h = map(int, (pred_pos[0]-current_size[0]/2, pred_pos[1]-current_size[1]/2, current_size[0], current_size[1]))
                roi_x1=max(0,x-w); roi_y1=max(0,y-h)
                roi_x2=min(frame.shape[1],x+w*2); roi_y2=min(frame.shape[0],y+h*2)
                roi_frame = frame[roi_y1:roi_y2, roi_x1:roi_x2]

                found_good_detection = False
                if roi_frame.size > 0:
                    roi_results = model(roi_frame, verbose=False, classes=[0], conf=CONFIDENCE_THRESHOLD)[0]
                    
                    if len(roi_results.boxes) > 0:
                        # پیدا کردن بزرگترین تشخیص در ناحیه
                        best_det_box = None; max_area = 0
                        for r in roi_results.boxes:
                            box = r.xyxy[0]; area = (box[2]-box[0])*(box[3]-box[1])
                            if area > max_area: max_area = area; best_det_box = box
                        
                        if best_det_box is not None:
                            global_w = int(best_det_box[2]-best_det_box[0])
                            global_h = int(best_det_box[3]-best_det_box[1])
                            
                            # شرط ۲: بررسی ابعاد برای جلوگیری از پرش‌های ناگهانی
                            last_area = current_size[0] * current_size[1]
                            if last_area > 0 and (global_w * global_h) > 0.25 * last_area:
                                global_x1 = int(best_det_box[0]+roi_x1); global_y1 = int(best_det_box[1]+roi_y1)
                                current_pos=(global_x1+global_w/2, global_y1+global_h/2)
                                current_size=(global_w, global_h)
                                kalman_state=np.array([current_pos[0],0,current_pos[1],0],dtype=np.float32)
                                patch = cv2.resize(frame[global_y1:global_y1+global_h, global_x1:global_x1+global_w], fixed_roi_size)
                                features = extract_channels(patch)
                                target_y_2d = create_gaussian_target(fixed_roi_size[::-1], fixed_roi_size[0]//2, fixed_roi_size[1]//2, 18.0)
                                model_A, model_B = teaching(features, target_y_2d, 0.01)
                                frames_since_update = 0
                                found_good_detection = True
                
                # اگر هیچ تشخیص خوبی پیدا نشد، به پیش‌بینی کالمن اکتفا کن
                if not found_good_detection:
                    current_pos = pred_pos # بازگشت به پیشنهاد شما

        if frames_since_update > MAX_FRAMES_TO_SKIP:
            tracking = False
        
        if tracking:
            if frames_since_update == 0:
                color = (0, 255, 0); pos_to_draw = current_pos; size_to_draw = current_size
            else:
                color = (0, 255, 0); pos_to_draw = pred_pos; size_to_draw = current_size
            
            x_draw = int(pos_to_draw[0]-size_to_draw[0]/2); y_draw = int(pos_to_draw[1]-size_to_draw[1]/2)
            w_draw = int(size_to_draw[0]); h_draw = int(size_to_draw[1])
            cv2.rectangle(frame, (x_draw, y_draw), (x_draw + w_draw, y_draw + h_draw), color, 2)
    
    fps_frame_count += 1
    if (time.time() - fps_start_time) >= 1.0:
        fps = fps_frame_count / (time.time() - fps_start_time)
        fps_frame_count = 0
        fps_start_time = time.time()

    cv2.putText(frame, f"FPS: {fps:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"PSR: {psr_score:.2f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0) if psr_score > 5.5 else (0,0,255), 2)

    cv2.imshow("Single Object Tracker", frame)
    out_writer.write(frame)
    if cv2.waitKey(1) & 0xFF == 27: break
        
cap.release()
out_writer.release()
cv2.destroyAllWindows()